In [1]:
import numpy as np
import heapq
from typing import List, Tuple, Set, Optional
import random
import time as time_module
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class TimeSpacePathPlanner:
    def __init__(self, grid_size: int = 100, weight_density: float = 0.1):
        self.grid_size = grid_size
        # Create 3D grid with random weights
        self.weights = np.zeros((grid_size + 1, grid_size + 1, grid_size + 1))

        # Randomly assign weights to some points
        num_points = int(weight_density * (grid_size + 1)**3)
        points = random.sample([(x, y, z)
                              for x in range(grid_size + 1)
                              for y in range(grid_size + 1)
                              for z in range(grid_size + 1)], num_points)

        for x, y, z in points:
            self.weights[x, y, z] = random.uniform(1.0, 5.0)

    def heuristic(self, a: Tuple[int, int, int], b: Tuple[int, int, int]) -> float:
        """Manhattan distance heuristic."""
        return abs(b[0] - a[0]) + abs(b[1] - a[1]) + abs(b[2] - a[2])

    def get_neighbors(self, point: Tuple[int, int, int]) -> List[Tuple[int, int, int]]:
        """Get valid neighboring points in the grid."""
        x, y, z = point
        neighbors = []

        for dx, dy, dz in [(0,0,1), (0,0,-1), (0,1,0), (0,-1,0), (1,0,0), (-1,0,0)]:
            new_x, new_y, new_z = x + dx, y + dy, z + dz
            if (0 <= new_x <= self.grid_size and
                0 <= new_y <= self.grid_size and
                0 <= new_z <= self.grid_size):
                neighbors.append((new_x, new_y, new_z))

        return neighbors

    def find_paths(self, paths_data: List[Tuple[Tuple[int, int, int], Tuple[int, int, int]]],
                   velocity: float, start_time: float = 0.0, timeout: float = 30.0) -> List[List[Tuple[int, int, int, float]]]:
        """Find non-intersecting paths for multiple start-end pairs."""
        all_paths = []
        occupied_space_time = set()
        start_process_time = time_module.time()

        for i, (start, end) in enumerate(paths_data):
            if time_module.time() - start_process_time > timeout:
                print(f"Timeout reached after processing {i} paths")
                break

            path = self.find_single_path(start, end, velocity, start_time,
                                       occupied_space_time, timeout - (time_module.time() - start_process_time))
            if path:
                all_paths.append(path)
                for x, y, z, t in path:
                    occupied_space_time.add((x, y, z, round(t, 3)))
            else:
                print(f"No valid path found for {start} to {end}")

        return all_paths

    def find_single_path(self, start: Tuple[int, int, int], end: Tuple[int, int, int],
                        velocity: float, start_time: float,
                        occupied_space_time: Set[Tuple[int, int, int, float]],
                        timeout: float) -> Optional[List[Tuple[int, int, int, float]]]:
        """Find shortest path avoiding time-space conflicts."""
        start_process_time = time_module.time()
        pq = [(self.heuristic(start, end), 0, start_time, start, [start])]
        visited = set()
        max_iterations = 110000
        iterations = 0

        while pq and iterations < max_iterations:
            if time_module.time() - start_process_time > timeout:
                print(f"Path finding timeout reached for {start} to {end}")
                return None

            iterations += 1
            _, cost, time, current, path = heapq.heappop(pq)

            if current == end:
                return [(x, y, z, t) for (x, y, z), t in zip(path,
                       np.linspace(start_time, time, len(path)))]

            if (current, round(time, 3)) in visited:
                continue

            visited.add((current, round(time, 3)))

            for next_point in self.get_neighbors(current):
                new_time = time + 1/velocity

                if (next_point[0], next_point[1], next_point[2], round(new_time, 3)) in occupied_space_time:
                    continue

                new_cost = cost + 1 + self.weights[next_point]
                f_score = new_cost + self.heuristic(next_point, end)
                heapq.heappush(pq, (f_score, new_cost, new_time, next_point, path + [next_point]))

        print(f"Path finding failed: exceeded {max_iterations} iterations")
        return None

    def plot_paths_plotly(self, paths: List[List[Tuple[int, int, int, float]]]):
        """Plot the paths in 3D using Plotly with interactive features."""
        if not paths:
            print("No paths to plot")
            return

        # Create figure
        fig = go.Figure()

        # Color scale for time progression
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'cyan']

        # Add paths
        for i, path in enumerate(paths):
            xs = [p[0] for p in path]
            ys = [p[1] for p in path]
            zs = [p[2] for p in path]
            times = [p[3] for p in path]

            # Add lines connecting path points
            fig.add_trace(go.Scatter3d(
                x=xs, y=ys, z=zs,
                line=dict(color=colors[i % len(colors)], width=4),
                mode='lines',
                name=f'Path {i+1}'
            ))

            # Add points along the path
            fig.add_trace(go.Scatter3d(
                x=xs, y=ys, z=zs,
                mode='markers',
                marker=dict(
                    size=4,
                    color=times,
                    colorscale='Viridis',
                    colorbar=dict(title='Time'),
                    showscale=True if i == 0 else False
                ),
                name=f'Points Path {i+1}'
            ))

            # Add start point (larger green marker)
            fig.add_trace(go.Scatter3d(
                x=[path[0][0]], y=[path[0][1]], z=[path[0][2]],
                mode='markers',
                marker=dict(size=8, color='green', symbol='diamond'),
                name=f'Start {i+1}'
            ))

            # Add end point (larger red marker)
            fig.add_trace(go.Scatter3d(
                x=[path[-1][0]], y=[path[-1][1]], z=[path[-1][2]],
                mode='markers',
                marker=dict(size=8, color='red', symbol='square'),
                name=f'End {i+1}'
            ))

        # Update layout for better visualization
        fig.update_layout(
            title='3D Time-Dependent Paths',
            scene=dict(
                xaxis_title='X',
                yaxis_title='Y',
                zaxis_title='Z',
                aspectmode='cube'
            ),
            showlegend=True,
            legend=dict(x=1.1, y=0.5),
            margin=dict(l=0, r=0, b=0, t=30)
        )

        # Show the interactive plot
        fig.show()



def get_valid_coordinate(prompt: str, max_value: int) -> int:
    """Helper function to get valid coordinate input."""
    while True:
        try:
            value = int(input(prompt))
            if 0 <= value <= max_value:
                return value
            print(f"Please enter a value between 0 and {max_value}")
        except ValueError:
            print("Please enter a valid integer")

def get_point(point_type: str, grid_size: int) -> Tuple[int, int, int]:
    """Get a 3D point from user input."""
    print(f"\nEnter coordinates for {point_type} point:")
    x = get_valid_coordinate(f"Enter x coordinate (0-{grid_size}): ", grid_size)
    y = get_valid_coordinate(f"Enter y coordinate (0-{grid_size}): ", grid_size)
    z = get_valid_coordinate(f"Enter z coordinate (0-{grid_size}): ", grid_size)
    return (x, y, z)

def main():
    # Get grid size from user
    while True:
        try:
            grid_size = int(input("Enter grid size (recommended 20-100): "))
            if grid_size > 0:
                break
            print("Please enter a positive integer")
        except ValueError:
            print("Please enter a valid integer")

    # Get number of paths from user
    while True:
        try:
            num_paths = int(input("Enter number of paths to calculate: "))
            if num_paths > 0:
                break
            print("Please enter a positive integer")
        except ValueError:
            print("Please enter a valid integer")

    # Initialize planner
    planner = TimeSpacePathPlanner(grid_size=grid_size, weight_density=0.1)

    # Get start and end points for each path
    paths_data = []
    for i in range(num_paths):
        print(f"\nPath {i+1}:")
        start_point = get_point("start", grid_size)
        end_point = get_point("end", grid_size)
        paths_data.append((start_point, end_point))

    # Display the input paths
    print("\nPaths to calculate:")
    for i, (start, end) in enumerate(paths_data, 1):
        print(f"Path {i}: {start} -> {end}")

    # Get velocity from user
    while True:
        try:
            velocity = float(input("\nEnter velocity (m/s, recommended 1.0): "))
            if velocity > 0:
                break
            print("Please enter a positive number")
        except ValueError:
            print("Please enter a valid number")

    # Confirm before proceeding
    proceed = input("\nReady to calculate paths. Press Enter to continue...")

    # Find paths with timeout
    print("\nCalculating paths...")
    paths = planner.find_paths(paths_data, velocity=velocity, start_time=0.0, timeout=30.0)

    if paths:
        print(f"\nFound {len(paths)} valid paths out of {len(paths_data)} requested")
        # Plot results using Plotly
        print("\nGenerating 3D visualization...")
        planner.plot_paths_plotly(paths)
    else:
        print("No valid paths found")

if __name__ == "__main__":
    main()

Enter grid size (recommended 20-100): 100
Enter number of paths to calculate: 3

Path 1:

Enter coordinates for start point:
Enter x coordinate (0-100): 0
Enter y coordinate (0-100): 0
Enter z coordinate (0-100): 20

Enter coordinates for end point:
Enter x coordinate (0-100): 10
Enter y coordinate (0-100): 10
Enter z coordinate (0-100): 100

Path 2:

Enter coordinates for start point:
Enter x coordinate (0-100): 0
Enter y coordinate (0-100): 20
Enter z coordinate (0-100): 0

Enter coordinates for end point:
Enter x coordinate (0-100): 10
Enter y coordinate (0-100): 100
Enter z coordinate (0-100): 10

Path 3:

Enter coordinates for start point:
Enter x coordinate (0-100): 20
Enter y coordinate (0-100): 0
Enter z coordinate (0-100): 0

Enter coordinates for end point:
Enter x coordinate (0-100): 100
Enter y coordinate (0-100): 10
Enter z coordinate (0-100): 10

Paths to calculate:
Path 1: (0, 0, 20) -> (10, 10, 100)
Path 2: (0, 20, 0) -> (10, 100, 10)
Path 3: (20, 0, 0) -> (100, 10, 10)